# **START SESSION**

In [ ]:
!pip install pyspark

In [1]:
import pyspark
from pyspark.ml.clustering import KMeans
from pyspark.ml.clustering import KMeansModel
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorIndexer

import matplotlib.pyplot as plt
import pandas as pd
import tempfile
import numpy as np
import seaborn as sns 
%matplotlib inline

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[2]") \
    .appName("Proyek PBD Cluster E-Commerce") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
spark

# **CONVERT AND LOAD DATA**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
data = spark.read.load("/content/drive/MyDrive/PROYEK PBD/dataset/data.csv",
                     format="csv", inferSchema="true", header="true")

Konversi format data yang sebelumnya berupa format CSV, menjadi Hadoop Format file dalam format **Parquet**.

In [ ]:
data.write.format("parquet").mode("overwrite").save("/content/drive/MyDrive/PROYEK PBD/dataset/data_parq")

In [5]:
data_parq = spark.read.load("/content/drive/MyDrive/PROYEK PBD/dataset/data_parq",
                     format="parquet", inferSchema="true", header="true")

# **EXPLORATORY DATA**

In [6]:
data_parq.show(10)
data_parq.printSchema()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [7]:
#check deskripsi dataset
data_parq.describe().show()

+-------+------------------+------------------+--------------------+------------------+---------------+-----------------+------------------+-----------+
|summary|         InvoiceNo|         StockCode|         Description|          Quantity|    InvoiceDate|        UnitPrice|        CustomerID|    Country|
+-------+------------------+------------------+--------------------+------------------+---------------+-----------------+------------------+-----------+
|  count|            541909|            541909|              540455|            541909|         541909|           541909|            406829|     541909|
|   mean|  559965.752026781|27623.240210938104|             20713.0|  9.55224954743324|           null|4.611113626082972|15287.690570239585|       null|
| stddev|13428.417280800133| 16799.73762842775|                null|218.08115785023486|           null| 96.7598530611797| 1713.600303321594|       null|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|            -8

# **DATA CLEANING AND DATA MANIPULATION**

**CLEANING DATA FROM NULL VALUES**

In [8]:
from pyspark.sql.functions import count
#check and remove the null values
def my_count(df_in):
    df_in.agg( *[ count(c).alias(c) for c in df_in.columns ] ).show()

In [9]:
my_count(data_parq)

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|   541909|   541909|     540455|  541909|     541909|   541909|    406829| 541909|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



Melalui proses penghitungan, ditemukan sejumlah baris data pada setiap kolom tidak sama, pada kolom **CustomerID** terdapat nilai null. Maka record data yang memiliki nilai null akan dihapus dari frame data dengan menggunakan fungsi .dropna()

In [10]:
ecom_data = data_parq.dropna(how='any')
my_count(ecom_data)

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|   406829|   406829|     406829|  406829|     406829|   406829|    406829| 406829|
+---------+---------+-----------+--------+-----------+---------+----------+-------+

